<h1 align="center">DATA MANIPULATION</h1> 

### Introduction

In this project, I will use SQL to manipulate football data from the different leagues around the world. The data will be extracted from database with various tables. The result will be the final log of the 2015/2016 Premier League season, which Leicester City won. All the data manipulation queries will written in SQL, but python will be used to execute them.

### Step 1: Import Packages

pandassql will help you use sql commands inside sql. sqlite3 will help you import a SQL database with different tables.

In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf
import sqlite3
import opendatasets as od

### Step 2: Import tables to dataframes

Download the database from Kaggle. Import the sql database with all the tables. Then convert the tables to dataframes.

In [2]:
con = sqlite3.connect("soccer/database.sqlite")

In [3]:
League = pd.read_sql('SELECT * FROM League', con)
Match = pd.read_sql('SELECT * FROM Match', con)
Team = pd.read_sql('SELECT * FROM Team', con)

### Step 3: Join Tables

For this exercise, you only want to use the England Premier League data, but you want all the data in one table. You will join 3 different tables to create one massive table with all the premier league data you need.

In [4]:
join = 'SELECT a.match_api_id AS match_id, \
               b.name AS league, a.season,  \
               a.date, c.team_long_name AS home_team, \
               d.team_long_name as away_team, \
               a.home_team_goal AS home_goal, \
               a.away_team_goal AS away_goal \
        FROM Match AS a \
        LEFT JOIN League AS b \
        ON a.country_id = b.country_id \
        LEFT JOIN Team AS c \
        ON a.home_team_api_id = c.team_api_id \
        LEFT JOIN Team AS d \
        ON a.away_team_api_id = d.team_api_id \
        WHERE a.country_id IN ("1729")'

df = sqldf(join)

In [5]:
df.head()

,match_id,league,season,date,home_team,away_team,home_goal,away_goal
0,489042,England Premier League,2008/2009,2008-08-17 00:00:00,Manchester United,Newcastle United,1,1
1,489043,England Premier League,2008/2009,2008-08-16 00:00:00,Arsenal,West Bromwich Albion,1,0
2,489044,England Premier League,2008/2009,2008-08-16 00:00:00,Sunderland,Liverpool,0,1
3,489045,England Premier League,2008/2009,2008-08-16 00:00:00,West Ham United,Wigan Athletic,2,1
4,489046,England Premier League,2008/2009,2008-08-17 00:00:00,Aston Villa,Manchester City,4,2


### Step 4: Clean Data

In this step you will do simple cleaning of the data so you can tell if a game resulted in a win, draw or loss. 

In [6]:
clean = 'SELECT *, CASE WHEN home_goal = away_goal then 1 else 0 end as draw \
                  ,CASE WHEN home_goal > away_goal then 1 else 0 end as home_win \
                  ,CASE WHEN home_goal < away_goal then 1 else 0 end as away_win \
         FROM df'

df = sqldf(clean)

In [7]:
df.head()

,match_id,league,season,date,home_team,away_team,home_goal,away_goal,draw,home_win,away_win
0,489042,England Premier League,2008/2009,2008-08-17 00:00:00,Manchester United,Newcastle United,1,1,1,0,0
1,489043,England Premier League,2008/2009,2008-08-16 00:00:00,Arsenal,West Bromwich Albion,1,0,0,1,0
2,489044,England Premier League,2008/2009,2008-08-16 00:00:00,Sunderland,Liverpool,0,1,0,0,1
3,489045,England Premier League,2008/2009,2008-08-16 00:00:00,West Ham United,Wigan Athletic,2,1,0,1,0
4,489046,England Premier League,2008/2009,2008-08-17 00:00:00,Aston Villa,Manchester City,4,2,0,1,0


### Step 5: Get Results Of Every North London Derby

In this step you will look at every result and date of the North London Derby since the 2008/2009 season. You will also add the name of the stadium the teams played in for every fixture. The Emirates Stadium is Arsenal's home stadium and White Hart Lane was Tottenham's home stadium at the time.

In [8]:
LondonDerby = 'SELECT  Season, \
                       SUBSTR(date,1,10) AS Date, \
                       CASE WHEN home_team IN ("Arsenal") THEN "Emirates Stadium" ELSE "White Hart Lane" \
                            END AS Stadium, \
                       home_team AS Home, \
                       (home_goal || "-" || away_goal) AS Score, \
                       away_team AS Away\
               FROM df \
               WHERE (home_team IN ("Arsenal") AND away_team IN  ("Tottenham Hotspur")) \
                      OR (home_team IN ("Tottenham Hotspur") AND away_team IN ("Arsenal")) \
               ORDER BY Season, Date'

sqldf(LondonDerby)

,season,Date,Stadium,Home,Score,Away
0,2008/2009,2008-10-29,Emirates Stadium,Arsenal,4-4,Tottenham Hotspur
1,2008/2009,2009-02-08,White Hart Lane,Tottenham Hotspur,0-0,Arsenal
2,2009/2010,2009-10-31,Emirates Stadium,Arsenal,3-0,Tottenham Hotspur
3,2009/2010,2010-04-14,White Hart Lane,Tottenham Hotspur,2-1,Arsenal
4,2010/2011,2010-11-20,Emirates Stadium,Arsenal,2-3,Tottenham Hotspur
5,2010/2011,2011-04-20,White Hart Lane,Tottenham Hotspur,3-3,Arsenal
6,2011/2012,2011-10-02,White Hart Lane,Tottenham Hotspur,2-1,Arsenal
7,2011/2012,2012-02-26,Emirates Stadium,Arsenal,5-2,Tottenham Hotspur
8,2012/2013,2012-11-17,Emirates Stadium,Arsenal,5-2,Tottenham Hotspur
9,2012/2013,2013-03-03,White Hart Lane,Tottenham Hotspur,2-1,Arsenal


### Step 6: Find Manchester United vs. Arsenal Score Aug 2011

In this step you will find the final score fo the Manchester United vs Arsenal game which took place on the 28th Of August 2011. The format will be similar to the one above.

In [9]:
MUNARS = 'SELECT  Season, \
                       SUBSTR(date,1,10) AS Date, \
                       CASE WHEN home_team IN ("Arsenal") THEN "Emirates Stadium" ELSE "Old Trafford" \
                            END AS Stadium, \
                       home_team AS Home, \
                       (home_goal || "-" || away_goal) AS Score, \
                       away_team AS Away\
               FROM df \
               WHERE home_team IN ("Manchester United") AND away_team IN  ("Arsenal") \
               AND date IN ("2011-08-28 00:00:00")\
               ORDER BY Season, Date'

sqldf(MUNARS)

,season,Date,Stadium,Home,Score,Away
0,2011/2012,2011-08-28,Old Trafford,Manchester United,8-2,Arsenal


### Step 7: Create Final Log For 2015/2016 Season

In this step you will recreate the final log of the 2015/2016 Premier League Season. You will first create the home log, then create the away log. Once both logs are created, you will join them to create the final log.

In [10]:
# Create Log For Home Fixtures
Home = 'SELECT DISTINCT home_team, \
                        SUM(CASE WHEN home_win = 1 THEN 3 ELSE 0 END) + SUM(draw) AS HomePoints, \
                        SUM(home_win) AS HomeWins, \
                        SUM(draw) AS Homedraws, \
                        SUM(CASE WHEN away_win = 1 THEN 1 ELSE 0 END) AS HomeLosses, \
                        SUM(home_goal) AS home_scored, \
                        SUM(away_goal* -1) AS home_conceded, \
                        SUM(home_goal) + SUM(away_goal* -1) AS HomeGoalDifference \
        FROM df \
        WHERE season IN ("2015/2016")\
        GROUP BY home_team \
        ORDER BY SUM(CASE WHEN home_win = 1 THEN 3 ELSE 0 END) + SUM(draw) DESC'

HomeLog = sqldf(Home)

In [11]:
# Create Log For Away Fixtures
Away = 'SELECT DISTINCT Away_team, \
                        SUM(CASE WHEN Away_win = 1 THEN 3 ELSE 0 END) + SUM(draw) AS AwayPoints, \
                        SUM(Away_win) AS AwayWins, \
                        SUM(draw) AS Awaydraws, \
                        SUM(CASE WHEN home_win = 1 THEN 1 ELSE 0 END) AS AwayLosses, \
                        SUM(Away_goal) AS Away_scored, \
                        SUM(home_goal* -1) AS Away_conceded, \
                        SUM(Away_goal) + SUM(home_goal* -1) AS AwayGoalDifference \
        FROM df \
        WHERE season IN ("2015/2016")\
        GROUP BY Away_team \
        ORDER BY SUM(CASE WHEN Away_win = 1 THEN 3 ELSE 0 END) + SUM(draw) DESC'

AwayLog = sqldf(Away)

In [12]:
# Combine Both Home & Away Fixtures
ALL = 'SELECT home_team AS Team, \
              (HomeWins + AwayWins + Homedraws + AwayDraws + HomeLosses + AwayLosses) AS Played, \
              (HomePoints + AwayPoints) AS Points,\
              (HomeWins + AwayWins) AS Wins, \
              (Homedraws + AwayDraws) AS Draws,\
              (HomeLosses + AwayLosses) AS Losses,\
              (home_scored + Away_Scored) AS Scored, \
              ((home_conceded + Away_conceded))*-1 AS Conceded,\
              (HomeGoalDifference + AwayGoalDifference) AS GoalDifference \
       FROM HomeLog \
       LEFT JOIN AwayLog \
       ON home_team = away_team \
       ORDER BY (HomePoints + AwayPoints) DESC'

Log = sqldf(ALL)
Log.index = Log.index + 1
Log

,Team,Played,Points,Wins,Draws,Losses,Scored,Conceded,GoalDifference
1,Leicester City,38,81,23,12,3,68,36,32
2,Arsenal,38,71,20,11,7,65,36,29
3,Tottenham Hotspur,38,70,19,13,6,69,35,34
4,Manchester United,38,66,19,9,10,49,35,14
5,Manchester City,38,66,19,9,10,71,41,30
6,Southampton,38,63,18,9,11,59,41,18
7,West Ham United,38,62,16,14,8,65,51,14
8,Liverpool,38,60,16,12,10,63,50,13
9,Stoke City,38,51,14,9,15,41,55,-14
10,Chelsea,38,50,12,14,12,59,53,6


________________________________